In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import requests

In [2]:
esg = pd.read_csv('data.csv')

In [3]:
esg.shape

(722, 21)

In [4]:
esg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722 entries, 0 to 721
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ticker                722 non-null    object
 1   name                  722 non-null    object
 2   currency              722 non-null    object
 3   exchange              722 non-null    object
 4   industry              709 non-null    object
 5   logo                  702 non-null    object
 6   weburl                706 non-null    object
 7   environment_grade     722 non-null    object
 8   environment_level     722 non-null    object
 9   social_grade          722 non-null    object
 10  social_level          722 non-null    object
 11  governance_grade      722 non-null    object
 12  governance_level      722 non-null    object
 13  environment_score     722 non-null    int64 
 14  social_score          722 non-null    int64 
 15  governance_score      722 non-null    in

In [5]:
esg.isna().sum()

ticker                   0
name                     0
currency                 0
exchange                 0
industry                13
logo                    20
weburl                  16
environment_grade        0
environment_level        0
social_grade             0
social_level             0
governance_grade         0
governance_level         0
environment_score        0
social_score             0
governance_score         0
total_score              0
last_processing_date     0
total_grade              0
total_level              0
cik                      0
dtype: int64

In [6]:
esg.head()

,ticker,name,currency,exchange,industry,logo,weburl,environment_grade,environment_level,social_grade,...,governance_grade,governance_level,environment_score,social_score,governance_score,total_score,last_processing_date,total_grade,total_level,cik
0,dis,Walt Disney Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,https://static.finnhub.io/logo/ef50b4a2b263c84...,https://thewaltdisneycompany.com/,A,High,BB,...,BB,Medium,510,316,321,1147,19-04-2022,BBB,High,1744489
1,gm,General Motors Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Automobiles,https://static.finnhub.io/logo/9253db78-80c9-1...,https://www.gm.com/,A,High,BB,...,B,Medium,510,303,255,1068,17-04-2022,BBB,High,1467858
2,gww,WW Grainger Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Trading Companies and Distributors,https://static.finnhub.io/logo/f153dcda-80eb-1...,https://www.grainger.com/,B,Medium,BB,...,B,Medium,255,385,240,880,19-04-2022,BB,Medium,277135
3,mhk,Mohawk Industries Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Consumer products,https://static.finnhub.io/logo/26868a62-80ec-1...,https://mohawkind.com/,A,High,B,...,BB,Medium,570,298,303,1171,18-04-2022,BBB,High,851968
4,lyv,Live Nation Entertainment Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,https://static.finnhub.io/logo/1cd144d2-80ec-1...,https://www.livenationentertainment.com/,BBB,High,BB,...,B,Medium,492,310,250,1052,18-04-2022,BBB,High,1335258


In [7]:
esg.ticker.nunique()

722

In [8]:
esg['last_processing_date'] = pd.to_datetime(esg['last_processing_date'],dayfirst=True)

In [39]:
!pip install yfinance


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     -------------------------------------- 108.7/108.7 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 3.0/3.0 MB 4.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for peewee: filename=peewee-3.17.9-py3-none-any.whl size=139096 sha256=8aedda9820e34f45330bea23dc3e956a355a9304e7c852d017e1fd9f61fa10f3
  Stored in directory: c:\users\shushant pc\appdata\local\pip\cache\wheels\af\56\87\585f494e912f4c33a5b6c475a48007d54e564ef1db92e24e84
Successfully built peewee


In [9]:
import yfinance as yf

class DataGathering:
    def __init__(self, df):
        self.df = df
        self.counter = 0
    def get_revenue_data(self, company_ticker):
        ticker = yf.Ticker(company_ticker)
        return ticker.financials

    def get_annual_eps(self,company_ticker,last_processing_date):
        data = self.get_revenue_data(company_ticker)
        self.counter = self.counter + 1
        if(self.counter%50==0):
            print(self.counter)
        try:
            for date in data.columns:
                if(date.year == last_processing_date.year):
                    return float(data[date]['Diluted EPS']), float(data[date]['Gross Profit'])
        except:
            print('Not found for ticker: ',company_ticker)
            return 0.0, 0.0

    def gather_data(self, column):
        self.df[column] = self.df.apply(lambda d: pd.Series(self.get_annual_eps(d['ticker'],d['last_processing_date'])), axis = 1)
        return self.df
        

In [10]:
gather = DataGathering(esg)
df = gather.gather_data(['annual_eps', 'gross_profit'])

Not found for ticker:  aame
Not found for ticker:  abos
Not found for ticker:  absi
Not found for ticker:  acet
Not found for ticker:  abcb
Not found for ticker:  achl
Not found for ticker:  aciu
Not found for ticker:  achv
Not found for ticker:  acnb
Not found for ticker:  act
Not found for ticker:  adag
Not found for ticker:  acxp
50
Not found for ticker:  adap
Not found for ticker:  adil
Not found for ticker:  acgl
Not found for ticker:  advm
Not found for ticker:  aemd
Not found for ticker:  afmd
Not found for ticker:  afcg
Not found for ticker:  afbi
Not found for ticker:  agnc
Not found for ticker:  aib
100
Not found for ticker:  akro
Not found for ticker:  aktx
Not found for ticker:  aldx
Not found for ticker:  alec
Not found for ticker:  algs
Not found for ticker:  allo
Not found for ticker:  bkng
Not found for ticker:  cinf
150
Not found for ticker:  ntrs
Not found for ticker:  allk
Not found for ticker:  nwsa
Not found for ticker:  pfg
200
Not found for ticker:  zion
Not foun

In [13]:
df.head()

,ticker,name,currency,exchange,industry,logo,weburl,environment_grade,environment_level,social_grade,...,environment_score,social_score,governance_score,total_score,last_processing_date,total_grade,total_level,cik,annual_eps,gross_profit
0,dis,Walt Disney Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,https://static.finnhub.io/logo/ef50b4a2b263c84...,https://thewaltdisneycompany.com/,A,High,BB,...,510,316,321,1147,2022-04-19,BBB,High,1744489,1.72,2.832100e+10
1,gm,General Motors Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Automobiles,https://static.finnhub.io/logo/9253db78-80c9-1...,https://www.gm.com/,A,High,BB,...,510,303,255,1068,2022-04-17,BBB,High,1467858,6.13,2.098100e+10
2,gww,WW Grainger Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Trading Companies and Distributors,https://static.finnhub.io/logo/f153dcda-80eb-1...,https://www.grainger.com/,B,Medium,BB,...,255,385,240,880,2022-04-19,BB,Medium,277135,30.06,5.849000e+09
3,mhk,Mohawk Industries Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Consumer products,https://static.finnhub.io/logo/26868a62-80ec-1...,https://mohawkind.com/,A,High,B,...,570,298,303,1171,2022-04-18,BBB,High,851968,0.39,2.943400e+09
4,lyv,Live Nation Entertainment Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,https://static.finnhub.io/logo/1cd144d2-80ec-1...,https://www.livenationentertainment.com/,BBB,High,BB,...,492,310,250,1052,2022-04-18,BBB,High,1335258,0.64,4.343730e+09


In [12]:
df.isna().sum()

ticker                   0
name                     0
currency                 0
exchange                 0
industry                13
logo                    20
weburl                  16
environment_grade        0
environment_level        0
social_grade             0
social_level             0
governance_grade         0
governance_level         0
environment_score        0
social_score             0
governance_score         0
total_score              0
last_processing_date     0
total_grade              0
total_level              0
cik                      0
annual_eps              83
gross_profit            84
dtype: int64

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722 entries, 0 to 721
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ticker                722 non-null    object        
 1   name                  722 non-null    object        
 2   currency              722 non-null    object        
 3   exchange              722 non-null    object        
 4   industry              709 non-null    object        
 5   logo                  702 non-null    object        
 6   weburl                706 non-null    object        
 7   environment_grade     722 non-null    object        
 8   environment_level     722 non-null    object        
 9   social_grade          722 non-null    object        
 10  social_level          722 non-null    object        
 11  governance_grade      722 non-null    object        
 12  governance_level      722 non-null    object        
 13  environment_score   

In [14]:
df.dropna(subset=['annual_eps','gross_profit'], inplace = True)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 638 entries, 0 to 721
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ticker                638 non-null    object        
 1   name                  638 non-null    object        
 2   currency              638 non-null    object        
 3   exchange              638 non-null    object        
 4   industry              635 non-null    object        
 5   logo                  626 non-null    object        
 6   weburl                629 non-null    object        
 7   environment_grade     638 non-null    object        
 8   environment_level     638 non-null    object        
 9   social_grade          638 non-null    object        
 10  social_level          638 non-null    object        
 11  governance_grade      638 non-null    object        
 12  governance_level      638 non-null    object        
 13  environment_score     638

In [16]:
df.to_csv('esd_eps_data.csv', index = False)

In [ ]:
import yfinance as yf
ticker = yf.Ticker('gs')
ticker.financials

In [4]:
dat = yf.Ticker("MSFT")

In [5]:
dat.info

{'address1': 'One Microsoft Way',
 'city': 'Redmond',
 'state': 'WA',
 'zip': '98052-6399',
 'country': 'United States',
 'phone': '425 882 8080',
 'website': 'https://www.microsoft.com',
 'industry': 'Software - Infrastructure',
 'industryKey': 'software-infrastructure',
 'industryDisp': 'Software - Infrastructure',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power 

In [6]:
dat.calendar

{'Dividend Date': datetime.date(2025, 3, 12),
 'Ex-Dividend Date': datetime.date(2025, 2, 19),
 'Earnings Date': [datetime.date(2025, 4, 23), datetime.date(2025, 4, 28)],
 'Earnings High': 3.56,
 'Earnings Low': 3.16,
 'Earnings Average': 3.22615,
 'Revenue High': 69897682310,
 'Revenue Low': 67140000000,
 'Revenue Average': 68534440260}

In [11]:
dat.analyst_price_targets

{'current': 408.21,
 'high': 650.0,
 'low': 420.0,
 'mean': 508.051,
 'median': 500.0}

In [13]:
dat.quarterly_income_stmt

,2024-12-31,2024-09-30,2024-06-30,2024-03-31,2023-12-31
Tax Effect Of Unusual Items,-203220000.0,57190000.0,-16263853.211009,-33660000.0,-29160000.0
Tax Rate For Calcs,0.18,0.19,0.191339,0.18,0.18
Normalized EBITDA,37915000000.0,37933000000.0,34416000000.0,33741000000.0,33556000000.0
Total Unusual Items,-1129000000.0,301000000.0,-85000000.0,-187000000.0,-162000000.0
Total Unusual Items Excluding Goodwill,-1129000000.0,301000000.0,-85000000.0,-187000000.0,-162000000.0
Net Income From Continuing Operation Net Minority Interest,24108000000.0,24667000000.0,22036000000.0,21939000000.0,21870000000.0
Reconciled Depreciation,6827000000.0,7383000000.0,6380000000.0,6027000000.0,5959000000.0
Reconciled Cost Of Revenue,21799000000.0,20099000000.0,19684000000.0,18505000000.0,19623000000.0
EBITDA,36786000000.0,38234000000.0,34331000000.0,33554000000.0,33394000000.0
EBIT,29959000000.0,30851000000.0,27951000000.0,27527000000.0,27435000000.0


In [17]:
dat.income_stmt

,2024-06-30,2023-06-30,2022-06-30,2021-06-30
Tax Effect Of Unusual Items,-99918000.0,-2850000.0,43754000.0,180160797.164637
Tax Rate For Calcs,0.182,0.19,0.131,0.138266
Normalized EBITDA,133558000000.0,105155000000.0,99905000000.0,83831000000.0
Total Unusual Items,-549000000.0,-15000000.0,334000000.0,1303000000.0
Total Unusual Items Excluding Goodwill,-549000000.0,-15000000.0,334000000.0,1303000000.0
Net Income From Continuing Operation Net Minority Interest,88136000000.0,72361000000.0,72738000000.0,61271000000.0
Reconciled Depreciation,22287000000.0,13861000000.0,14460000000.0,11686000000.0
Reconciled Cost Of Revenue,74114000000.0,65863000000.0,62650000000.0,52232000000.0
EBITDA,133009000000.0,105140000000.0,100239000000.0,85134000000.0
EBIT,110722000000.0,91279000000.0,85779000000.0,73448000000.0


In [14]:
dat.history(period='1mo')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-01-22 00:00:00-05:00,437.559998,447.269989,436.000000,446.200012,27803800,0.0,0.0
2025-01-23 00:00:00-05:00,442.000000,446.750000,441.500000,446.709991,18389300,0.0,0.0
2025-01-24 00:00:00-05:00,445.160004,446.649994,441.399994,444.059998,15549500,0.0,0.0
2025-01-27 00:00:00-05:00,424.010010,435.200012,423.500000,434.559998,35647800,0.0,0.0
2025-01-28 00:00:00-05:00,434.600006,448.380005,431.380005,447.200012,23491700,0.0,0.0
2025-01-29 00:00:00-05:00,446.690002,446.880005,440.399994,442.329987,23581400,0.0,0.0
2025-01-30 00:00:00-05:00,418.769989,422.859985,413.160004,414.989990,54586300,0.0,0.0
2025-01-31 00:00:00-05:00,418.980011,420.690002,414.910004,415.059998,34223400,0.0,0.0
2025-02-03 00:00:00-05:00,411.600006,415.410004,408.660004,410.920013,25679100,0.0,0.0
